In [1]:
import torch

In [11]:
class MyModule(torch.nn.Module):
    def __init__(self, N, M):
        super(MyModule, self).__init__()
        self.linear = torch.nn.Linear(N, M)

    def forward(self, input):
        return self.linear(input)


In [ ]:
my_module = MyModule(10,20)
# sm = torch.jit.script(my_module)
sm = torch.jit.script(my_module)
sm.save("models/my_module.pt")

In [ ]:
# sm.save("models/my_module.pt")

RuntimeError: Parent directory models does not exist.

In [ ]:
m = torch.jit.load("models/my_module.pt")

In [17]:
m

RecursiveScriptModule(
  original_name=MyModule
  (linear): RecursiveScriptModule(original_name=Linear)
)

In [20]:
x = torch.randn(10)

In [21]:
m(x)

tensor([-0.5386,  0.6545,  0.4650, -0.3320,  0.2735,  0.2796, -0.4549,  0.2646,
        -0.9322, -0.3031, -0.3441, -0.3761,  0.6457,  0.6456, -0.2478, -0.2270,
         0.8485,  0.9710, -0.0596,  0.6110], grad_fn=<ViewBackward0>)

In [22]:
# sm = torch.jit.script(style_model.to(torch.float32))
# sm.save(f"models/{model_name}_float32.pt")

# sm = torch.jit.script(style_model.to(torch.float16))
# sm.save(f"models/{model_name}_float16.pt")


In [ ]:
# torch::Tensor sobel_dx = torch::tensor({{-1, 0, 1},
#                                     {-2, 0, 2},
#                                     {-1, 0, 1}}).to(torch::kFloat32);
# torch::Tensor sobel_dy = torch::tensor({{-1, -2, -1},
#                                     {0, 0, 0},
#                                     {1, 2, 1}}).to(torch::kFloat32);

# torch::Tensor sobel_kernel = torch::cat({sobel_dx, sobel_dy}, 0).unsqueeze(0).unsqueeze(0);



In [32]:
class Sobel(torch.nn.Module):
    def __init__(self):
        super(Sobel, self).__init__()
        sobel_dx = torch.tensor([[-1, 0, 1],
                                [-2, 0, 2],
                                [-1, 0, 1]]).to(torch.float16)
        sobel_dy = torch.tensor([[-1, -2, -1],
                                [0, 0, 0],
                                [1, 2, 1]]).to(torch.float16)
        sobel_kernel = torch.cat((sobel_dx, sobel_dy), 0).unsqueeze(0).unsqueeze(0)
        sobel_kernel = sobel_kernel.to(torch.float16)
        self.sobel_kernel = torch.nn.Parameter(sobel_kernel, requires_grad=False)
        self.sobel_cnn = torch.nn.Conv2d(3, 3, kernel_size=3, stride=1, padding=1, bias=False)
        self.sobel_cnn.weight = torch.nn.Parameter(sobel_kernel, requires_grad=False)

    def forward(self, x):
        return self.sobel_cnn(x)

sobel = Sobel().to(torch.float16)
sobel(torch.randn(3, 1428, 1904).to(torch.float16))
sm = torch.jit.script(sobel)
sm.save("models/sobel.pt")


RuntimeError: Given groups=1, weight of size [1, 1, 6, 3], expected input[1, 3, 1428, 1904] to have 1 channels, but got 3 channels instead

In [57]:
sobel_dx = torch.tensor([[-1, 0, 1],
                        [-2, 0, 2],
                        [-1, 0, 1]]).to(torch.float16)
print(sobel_dx.shape)
sobel_dy = torch.tensor([[-1, -2, -1],
                        [0, 0, 0],
                        [1, 2, 1]]).to(torch.float16)
print(sobel_dy.shape)
sobel_kernel = torch.cat([sobel_dx.unsqueeze(0), sobel_dy.unsqueeze(0)], 0)
print(sobel_kernel.shape)
X = torch.randn(1,3, 1428, 1904).to(torch.float16)

# sobel_cnn = torch.nn.Conv2d(3, 3, kernel_size=3, stride=1, padding=1, bias=False)
# sobel_cnn.weight = torch.nn.Parameter(sobel_kernel, requires_grad=False)
# sobel_cnn = sobel_cnn.to(torch.float16)
# sobel_cnn(X)

Y = torch.nn.functional.conv2d(X, sobel_kernel, stride=1, padding=1)

torch.Size([3, 3])
torch.Size([3, 3])
torch.Size([2, 3, 3])


RuntimeError: expected stride to be a single integer value or a list of 1 values to match the convolution dimensions, but got stride=[1, 1]